In [ ]:
import matplotlib
import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd
import os
import umap
import datashader as ds
import colorcet as cc

from dredFISH.Analysis import TissueGraph
from dredFISH.Visualization import Viz
from dredFISH.Utils.__init__plots import * 
from dredFISH.Utils import powerplots
import importlib
importlib.reload(Viz)
importlib.reload(TissueGraph)

#### Load data

In [ ]:
respath = '/bigstore/GeneralStorage/fangming/projects/dredfish/figures/'

In [ ]:
basepth = '/bigstore/GeneralStorage/Data/dredFISH/Dataset1'
!ls -alhtr $basepth
!head $basepth"/TMG.json"

In [ ]:
TMG = TissueGraph.TissueMultiGraph(basepath=basepth, 
                                   redo=False, # load existing 
                                  )
TMG

In [ ]:
# spatial coordinates
XY = TMG.Layers[0].XY
x, y = XY[:,0], XY[:,1]
###
x, y = y, x # a temporary hack
###

# measured basis
ftrs_mat = TMG.Layers[0].feature_mat

# types

# regions

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))
ax.boxplot(ftrs_mat)
# ax.set_ylim([-3,10])
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))
ax.boxplot(ftrs_mat)
ax.set_ylim([-3,10])
plt.show()

In [ ]:
%%time
fig, axs = plt.subplots(4, 6, figsize=(6*5, 4*4))
for i in range(24):
    ax = axs.flat[i]
    ax.scatter(x, y, c=ftrs_mat[:,i], s=0.5, cmap='jet', vmin=-3, vmax=3)
    ax.set_aspect('equal')
    ax.axis('off')
plt.show()
    

# 24 bits

In [ ]:
df = pd.DataFrame()
df['x'] = x
df['y'] = y
for i in range(24):
    df[f'b{i}'] = ftrs_mat[:,i]
df

In [ ]:
%%time
path = os.path.join(respath, 'basis_space.pdf')
P = powerplots.PlotScale(df['x'].max()-df['x'].min(), 
                         df['y'].max()-df['y'].min(),
                         # npxlx=300,
                         pxl_scale=20,
                        )
print(P.npxlx, P.npxly)

nx, ny = 6, 4
fig, axs = plt.subplots(ny, nx, figsize=(nx*6, ny*5))
for i in range(24):
    ax = axs.flat[i]
    aggdata = ds.Canvas(P.npxlx, P.npxly).points(df, 'x', 'y', agg=ds.mean(f'b{i}'))
    ax.imshow(aggdata, origin='lower', aspect='equal', cmap='jet', vmin=-3, vmax=3, interpolation='none')
    ax.set_title(f'b{i}', loc='left', y=0.9)
    ax.axis('off')
fig.subplots_adjust(wspace=0.05, hspace=0)
powerplots.savefig_autodate(fig, path)
plt.show()

# UMAP across 24 bits

In [ ]:
%%time
umap_mat = umap.UMAP(n_neighbors=30, min_dist=0.1).fit_transform(ftrs_mat)
umap_mat.shape


In [ ]:
df['umap_x'] = umap_mat[:,0]
df['umap_y'] = umap_mat[:,1]

In [ ]:
fig, ax = plt.subplots()
ax.scatter(umap_mat[:,0], umap_mat[:,1], s=0.1)
ax.axis('off')
plt.show()

In [ ]:
%%time
x, y = 'umap_x', 'umap_y'
path = os.path.join(respath, 'basis_umap.pdf')
P = powerplots.PlotScale(df[x].max()-df[x].min(), 
                         df[y].max()-df[y].min(),
                         npxlx=300,
                        )
print(P.npxlx, P.npxly)

nx, ny = 6, 4
fig, axs = plt.subplots(ny, nx, figsize=(nx*5, ny*4))
for i in range(24):
    ax = axs.flat[i]
    aggdata = ds.Canvas(P.npxlx, P.npxly).points(df, x, y, agg=ds.mean(f'b{i}'))
    ax.imshow(aggdata, origin='lower', aspect='equal', cmap='jet', vmin=-3, vmax=3, interpolation='none')
    ax.set_title(f'b{i}', loc='left', y=0.9)
    ax.axis('off')
fig.subplots_adjust(wspace=0.05, hspace=0.1)
powerplots.savefig_autodate(fig, path)
plt.show()

# Cell types (Leiden clustering)

# Local env (Isozones)
- same cell type zones
- instead quantify the nearest neighbors of the same cell type

# Local env (brain regions)